# Riccati transformed inner problem

This notebook will illustrate how to use solve the Riccati transformation of the inner problem to determine optimal shift $\ell$ and smoothing $\delta$ parameters for normalized mask functions $\Gamma$.

Specifically, for a normalized mask function with support $-c < x < c$, we solve the Riccati equation

$$ R' + R^2 = \delta^2 \Gamma,$$

with initial condition

$$ R(-c) = \delta,$$

to find the optimal shift $\ell^*$ as

$$\ell^*(\delta) = \left(\frac{1}{R(c)} - c\right) \delta.$$

We will solve this problem numerically using python for a compact error function mask.

It is possible to apply Newton iteration to the equation to efficiently determine the optimal smoothing that requires zero shift. This is done by differentiating the equations with respect to the parameter $\delta$.

# Imports

In [ ]:
import numpy as np
import scipy.integrate as spint
from scipy.special import erf
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

# Optimal shift

In [ ]:
def Γ(z):
    """Define the normalized mask function you want."""
    return 0.5*(1-erf(np.sqrt(np.pi)*z/np.sqrt(1-z**2)))

In [ ]:
def Xt(X, t, δ, Γ):
    """The derivative for the coupled Riccati equations."""
    return np.array([-X[0]**2 + δ**2*Γ(t), 
                   -2*X[1]*X[0] + 2*δ*Γ(t)])

In [ ]:
def solve_riccati(δ,Γ,ts=[-1,1]):
    """Solve the Riccati equation with mask K and damping scaling κ."""
    X0 = np.array([δ,1.])
    Xs = spint.odeint(Xt, X0, ts, tcrit=[-1,1],args=(δ,Γ))
    return Xs

In [ ]:
def shift(δ,Γ,ϵ=1e-16):
    """Calculate required shift given mask K and damping scaling κ."""
    R1, dR1 = solve_riccati(δ,Γ)[-1,:]
    return 1/R1 - 1, -dR1/R1**2

In [ ]:
shift(3.14,Γ)

In [ ]:
def ideal_shift(Γ,δ0,tol=1e-10,maxits=100):
    """Use Newton iteration to determine zero-shift smoothing δ."""
    δ,dδ = np.array([δ0]), np.array([1.])
    diff, its = 1, 0
    while diff > tol and its < maxits:
        li, dli = shift(δ[-1],Γ)
        diff, its = - li/dli, its+1
        δ, dδ = np.append(δ,δ[-1]+diff), np.append(dδ,diff)
    return δ, dδ

In [ ]:
ideal_shift(Γ,1)